# Planning for College
An analysis of how the price of university has changed over time along with a comparison of different possible investment portfolios that families could use to save for their children to go to college.
### Goals
* Use historical tuition data to predict the price of tuition for the year a child will start college
* Assuming an initial investment of a certain dollar amount, use monte carlo simulations to predict how savings portfolios of different weights will compare to each other
* Given the results of these simulations, provide a framework for families to set expectations for their children for how much of their tuition will be covered by the family's savings, and how much of it will have to come from outside sources such as loans, grants, and/or scholarships.

### Research questions
* How much has the price of university changed over the past 60 years after accounting for inflation?
* How much can we expect the price to increase each year?
* What is the average difference in price between public and private university?
* With an initial investment of *x* dollars, how much can a family expect to have saved after *x* years for each portfolio? How does that compare to the predicted cost of public and private university after *x* years?
* Is one portfolio better than another in the given scenario?

In [15]:
#import the dependecies for the project
import pandas as pd
import numpy as np
from pathlib import Path
import os
import alpaca_trade_api as tradeapi
from stock_data_analysis.MCForecastTools import MCSimulation
import hvplot.pandas
import altair as alt
from dotenv import load_dotenv
%matplotlib inline

## Part 1: Analyzing the Tuition Data
In this section, we will prepare and analyze data taken from the [National Center for Education Statistics](https://nces.ed.gov/programs/digest/d21/tables/dt21_330.10.asp).

### Which data to use for the analysis?
The initial DataFrame that we created after importing the CSV file contained a variety pricing data, such as the costs of tuition, fees, and room and board, as well as all of those costs together. For the purpose of this project, we decided only to use the data for tuition, fees, and room and board together. In addition, the file provided data showing the actual dollar amount for the years provided, as well as those prices adjusted for inflation. In order to create more accurate predicitons, we decided to use only the inflation-adjusted data for our analysis.

The notebook `separating_dataframes.ipynb` shows the process of separating and cleaning the data from the initial CSV file into three different DataFrames, then saving those DataFrames to their own CSV files to be used in our other notebooks. These DataFrames contain average pricing data for all public institutions, all private institutions, and both types together.

In [2]:
#read in CSV files for each type of institution
public_df = pd.read_csv(
    Path('./tuition_price_analysis/Resources/public_institutions.csv'),
    index_col='Year'
)

all_institutions_df = pd.read_csv(
    Path('./tuition_price_analysis/Resources/all_institutions.csv'),
    index_col='Year'
)

all_private_df = pd.read_csv(
    Path('./tuition_price_analysis/Resources/all_private_institutions.csv'),
    index_col='Year'
)

In [3]:
#display heads for each DataFrame
display(public_df.head())
display(all_private_df.head())
display(all_institutions_df.head())

,All Institutions,4-year,2-year
Year,,,
1963-64,7785,7923,5375
1968-69,8201,8433,6515
1969-70,8336,8624,6625
1970-71,8453,8783,6611
1971-72,8614,8984,6862


,All Institutions,4-year,2-year
Year,,,
1963-64,15486,15441,11203
1968-69,17124,17383,13841
1969-70,17607,17825,13884
1970-71,18078,18247,13931
1971-72,18560,18664,13979


,All Institutions,4-year,2-year
Year,,,
1963-64,10648,10973,6616
1968-69,10762,11401,7769
1969-70,10868,11665,7589
1970-71,10948,11820,7422
1971-72,11066,12007,7493


There was a formatting error in the index for the 1985-1986 year, so we edited the index to fix the error.

In [4]:
#correct the year 1985-86 for all dataframes
index = public_df.index              
index_list = index.tolist()
index_list[18] = '1985-86'

public_df.index=index_list
public_df.index.name='Year'

all_private_df.index=index_list
all_private_df.index.name='Year'

all_institutions_df.index=index_list
all_institutions_df.index.name='Year'

For this analysis, we chose to only analyze the pricing data for a traditional four-year education.

In [5]:
#drop the 'all institutions' and '2-year' columns to focus only on 4-year expenses.
all_institutions_4_year = all_institutions_df.drop(columns=['All Institutions', '2-year'])

private_4_year = all_private_df.drop(columns=['All Institutions', '2-year'])

public_4_year = public_df.drop(columns=['All Institutions', '2-year'])
public_4_year.head()

,4-year
Year,
1963-64,7923
1968-69,8433
1969-70,8624
1970-71,8783
1971-72,8984


We then concatenate the 4-year data from each type into one DataFrame

In [6]:
#concatenate the dataframes
all_tuition_df = pd.concat(
    [all_institutions_4_year, private_4_year, public_4_year],
    axis='columns',
    join='inner')

#rename columns
columns = ['All Institutions', 'Private', 'Public']
all_tuition_df.columns=columns
all_tuition_df.tail()

,All Institutions,Private,Public
Year,,,
2016-17,28838,44967,21134
2017-18,29013,45743,21263
2018-19,28121,46414,21401
2019-20,29436,46982,21520
2020-21,29033,46313,21337


The following visualization shows the change over time for each type.

In [7]:
all_tuition_df.hvplot(
    xlabel='School Year',
    ylabel='Total Tuition/Fees/Room and Board',
    title='Total Average University Expenses by Year - Adjusted for Inflation',
    rot=90
).opts(
    height=350,
    width=900
)

:NdOverlay   [Variable]
   :Curve   [Year]   (value)

Now that we have the pricing data for all of the institutions in the same dataframe, we can calculate the percent change for each year.

In [8]:
#create a dataframe for the percentage changes for each institution type
tuition_pct_change = all_tuition_df.pct_change().dropna()
tuition_pct_change.head()

,All Institutions,Private,Public
Year,,,
1968-69,0.039005,0.125769,0.064370
1969-70,0.023156,0.025427,0.022649
1970-71,0.013288,0.023675,0.018437
1971-72,0.015821,0.022853,0.022885
1972-73,0.039644,0.018163,0.062890


We can visualize the percent changes for each year using bar charts:

In [9]:
#chart configuration for all institutions
alt.Chart(tuition_pct_change.reset_index()).mark_bar().encode(
    x='Year',
    y=alt.Y('All Institutions', title='Percent Change')
).properties(
    title='Percent Change Per Year - All Institutions'
).configure_title(
    'center',
    fontSize=16
)

alt.Chart(...)

In [10]:
#chart configuration for private institutions
alt.Chart(tuition_pct_change.reset_index()).mark_bar().encode(
    x='Year',
    y=alt.Y('Private', title='Percent Change')
).properties(
    title='Percent Change Per Year - Private Institutions'
).configure_title(
    'center',
    fontSize=16
)

alt.Chart(...)

In [11]:
#chart configuration for public institutions
alt.Chart(tuition_pct_change.reset_index()).mark_bar().encode(
    x='Year',
    y=alt.Y('Public', title='Percent Change')
).properties(
    title='Percent Change Per Year - Public Institutions'
).configure_title(
    'center',
    fontSize=16
)

alt.Chart(...)

We can also use the `describe()` function to gain insight into these changes.

These statistics tell us that, overall, the price of tuition between all institutions increased by around 2 percent each year. Since the goal of our project is to give families a ballpark estimate of how much they can expect college to cost after a certain number of years, we can calculate the estimated cost by assuming a 2 percent increase for each year.

In [12]:
tuition_pct_change.describe()

,All Institutions,Private,Public
count,53.000000,53.000000,53.000000
mean,0.018866,0.021298,0.019259
std,0.026222,0.027375,0.028221
min,-0.061418,-0.048834,-0.071270
25%,0.009780,0.011358,0.009323
50%,0.022059,0.020849,0.022649
75%,0.036244,0.030422,0.035827
max,0.069243,0.125769,0.067389


Assuming this 2 percent increase and using the most recent pricing data as a starting point, we can use the Future Value formula to calculate a price estimate over a certain number of years. In the following example, we use this formula to estimate the total costs for a public institution in 2031.

In [13]:
#a function to calculate the future price
def calculate_future_price(current_price, number_of_years):
    future_price = current_price * (1 + .02) ** number_of_years
    return future_price

#call the function with a starting value of 21337 and a period of 10 years.
price_2031 = calculate_future_price(21337, 10)
price_2031

26009.683939428138

## Part 2: Comparing Portfolios
In simulating these different stock portfolios, our goal is to provide an alternative to a traditional savings account that would allow families to invest in their children's future education while mitigating risk. We decided to keep these portfolios simple by composing them of two Exchange-Traded Funds (ETFs), the SPDR S&P 500 ETF Trust (SPY), and the iShares Core US Aggregate Bond ETF (AGG). By providing Monte Carlo simulations of different weights, we are able to provide a tool for families to make an informed decision on which type of portfolio would best meet their needs.

### Collecting the Data
We used the Alpaca Trade API to collect data for the two ETFs and calculate the daily returns for each. We then used that daily return data as the basis for our Monte Carlo simulations.

In [16]:
#load the API credentials from the environment file
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")